# Why this code exists
This notebook takes the final annotations dataset **final_annotations_set.csv**, and tries to train ML models to learn the annotation pattern.

We first fix the annotations for tools.wmflabs.org references, as seen from the inaccessibility study.

We then enrich the dataset by removing some bad features and adding better features / categorizing existing ones.

We then run it through some models.

In [1]:
import json
import copy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import requests

In [2]:
final_annotation_set = pd.read_csv('data/final/all_aggregated_annotations/final_annotation_set.csv')
final_annotation_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   reference_id           2310 non-null   object 
 1   url                    2310 non-null   object 
 2   language_crawl         2310 non-null   object 
 3   statement_node         2310 non-null   object 
 4   subject                2310 non-null   object 
 5   predicate              2310 non-null   object 
 6   object                 2310 non-null   object 
 7   url_type               2310 non-null   object 
 8   is_inferred_from       2310 non-null   object 
 9   stated_in              2310 non-null   object 
 10  external_id_prop       2310 non-null   object 
 11  external_id            2310 non-null   object 
 12  internal_urls          2310 non-null   object 
 13  external_url           2310 non-null   object 
 14  wikimedia_import_urls  2310 non-null   object 
 15  retr

In [17]:
final_annotation_set[final_annotation_set.apply(lambda x: 'tools.wmflabs.org' in x['url'], axis=1)][final_annotation_set['language_crawl'] == 'es']['is_present'].value_counts()

/home/gabriela/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


1    32
0     4
Name: is_present, dtype: int64

# Correct tools.wmflabs.org author/publisher annotation

In [18]:
for i in final_annotation_set[final_annotation_set.apply(lambda x: 'tools.wmflabs.org' in x['url'], axis=1)].index:
    if final_annotation_set.loc[i,'author'] == 3:
        final_annotation_set.loc[i,'author'] = 2
    if final_annotation_set.loc[i,'publisher'] == 5:
        final_annotation_set.loc[i,'publisher'] = 4    
    if final_annotation_set.loc[i,'long_sub_publisher'] == 5.0:
        final_annotation_set.loc[i,'long_sub_publisher'] = 4.0        
    if final_annotation_set.loc[i,'authoritative'] == "Inaccessible":
        final_annotation_set.loc[i,'authoritative'] = "No"

# Remove non-informative and correlated features